**EDA**

Basic EDA to see what the data look like. 


Start by looking at the daily resistivity .csv files exactacted into Thr 20 process by 0.1_ExtractGzipFiles. 

In [0]:
import pandas as pd 
import glob

# Define file path to daily df #1
file_path = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/Thr 20 process/*Z"

# Read the csv into a pandas df
for file in glob.glob(file_path):
    df = pd.read_csv(file, header = None)
    # Display some basic info about the df 
    df.info()
    # Look at head of df
    display(df.head(10))


It's HUGE! 

Now, lets do some QC:

In [0]:
# Check for NaNs
print('NaNs')
print(df.isnull().sum())

# Check for zeros
print()
print('Zeros')
print(df.isin([0]).sum())

# Check for duplicate rows
print()
print(df[df.duplicated()])



Main issue is our duplicates. I think that is because sometimes the sample rate is so rapid its taking the same measurement in a matter of miliseconds. We could get rid of these when the time and the resisitivity reading is the same. 

So now remove duplicates and save as a filtered df. 

In [0]:
import os

# Define file paths
file_pattern = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/*Z"
output_folder = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/DuplicateFree" 

os.makedirs(output_folder, exist_ok=True)

# Remove duplicates from each file
for file in glob.glob(file_pattern):
    df = pd.read_csv(file, header = None)
    dfd = df.drop_duplicates().reset_index(drop=True)
    
    # Create new filename based on original file
    filename = os.path.basename(file).replace("Z", "Z_no_duplicate")
    save_path = os.path.join(output_folder, filename)
    
    #Save
    dfd.to_csv(save_path, index=False)

print("All datasets labelled and saved.")



Now let's take a wee look at the labelled fish data and how this is structured. 

In [0]:
# Define file path to daily df #1
file_path = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/Thr 20 process/test_KMThu16_2021_07.csv"
lfd = pd.read_csv(file_path, header = 0)


# Display some basic info about the df 
lfd.info()

# Look at head of df
display(lfd)



There are two types of events, event and LF NOISE. We only want the events because LF NOISE is something other than a fish passage event (i.e., ice, canoe, otter).

In [0]:
# Filter out so only have Event 
filtered_df = lfd[lfd['Etype'] == '<##> EVENT  ']

# Save df
filtered_df.to_csv("/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/Thr 20 process/test_KMThu16_2021_07_eventonly.csv", index=False)

# Investigate
display(filtered_df)
filtered_df.info()
filtered_df.describe()
